# Caso Práctico: Auditoría de Sesgos y Control de Salida para un Asistente de Contratación

Empresa  TalentTech HR Contexto: TalentTech está desarrollando un sistema de IA para pre-filtrar currículums y redactar descripciones de puestos de trabajo. Para evitar litigios y garantizar la diversidad, el sistema debe ser auditado para eliminar sesgos de género y suposiciones implícitas.

El Problema a Auditar (Sesgo Implícito): El modelo base de LLM (Large Language Model) tiende a asociar profesiones técnicas con pronombres masculinos y profesiones asistenciales con pronombres femeninos (ej. Asume que "el CEO" es un hombre y "la enfermera" es una mujer, tal como se menciona en tu material de estudio).

## Tarea de Prompt Engineering: Mitigación de Sesgos y Guardrails
Tu objetivo es diseñar dos prompts maestros que obliguen al LLM a romper su sesgo y a controlar su salida para un escenario crítico.

### Escenario 1: Mitigación de Sesgos (Técnica de Refutación / Neutralidad)
Input del Usuario (Intencionalmente Sesgado):

"El nuevo ingeniero jefe de software visitó al mejor enfermero del hospital para su examen físico."

Objetivo del Prompt: Forzar al modelo a neutralizar la frase y refutar cualquier suposición de género en las profesiones.

Requisitos del Prompt Maestro:

1. Instrucción de Neutralización: Pide al LLM que reescriba la frase de input usando pronombres y términos de género neutro (ej. "la persona que ejerce la ingeniería de software").

2. Técnica de Refutación: Incluye una instrucción explícita que obligue al modelo a justificar por qué el uso de un pronombre específico es problemático o sesgado.

3. Output Deseado (Generación): Una frase reescrita de manera neutral y una breve justificación.

In [3]:
# Configuración inicial con Google Gemini
import os
import google.generativeai as genai
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

# Configurar Gemini con tu API key
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# Inicializar el modelo Gemini 2.5 Flash
model = genai.GenerativeModel('gemini-2.5-flash')

# Texto de entrada intencionalmente sesgado
texto_sesgado = "El nuevo ingeniero jefe de software visitó al mejor enfermero del hospital para su examen físico."

# Prompt Maestro para Escenario 1: Mitigación de Sesgos
prompt_escenario_1 = f"""
Eres un auditor de sesgos lingüísticos especializado en identificar y neutralizar suposiciones de género en textos profesionales.

TAREA:
1. Analiza el siguiente texto e identifica cualquier suposición de género implícita en las profesiones mencionadas.
2. Reescribe el texto usando lenguaje completamente neutro en cuanto a género (usa términos como "la persona que...", "quien ejerce...", o artículos neutros).
3. Proporciona una justificación detallada explicando por qué el uso de pronombres específicos de género en el texto original es problemático y perpetúa estereotipos.

TEXTO A ANALIZAR:
"{texto_sesgado}"

FORMATO DE RESPUESTA:
**Texto Original Analizado:**
[Identifica las suposiciones de género]

**Texto Reescrito (Neutro):**
[Versión neutralizada del texto]

**Justificación del Sesgo:**
[Explica por qué el texto original es problemático y cómo perpetúa estereotipos de género en profesiones]
"""

# Llamar a la API de Gemini
response_1 = model.generate_content(prompt_escenario_1)

print("="*80)
print("ESCENARIO 1: MITIGACIÓN DE SESGOS")
print("="*80)
print(f"\nTexto Original:\n{texto_sesgado}\n")
print("-"*80)
print(f"\nRespuesta de Gemini 2.5 Flash:\n{response_1.text}")
print("="*80)

ESCENARIO 1: MITIGACIÓN DE SESGOS

Texto Original:
El nuevo ingeniero jefe de software visitó al mejor enfermero del hospital para su examen físico.

--------------------------------------------------------------------------------

Respuesta de Gemini 2.5 Flash:
**Texto Original Analizado:**
El texto original asume implícitamente el género de las personas que ocupan las profesiones mencionadas mediante el uso de artículos y sustantivos masculinos:
*   **"El nuevo ingeniero jefe de software":** El uso del artículo masculino "El" junto con el sustantivo "ingeniero" (en su forma masculina) asume que la persona que ocupa el cargo de ingeniero jefe de software es un hombre.
*   **"el mejor enfermero del hospital":** De manera similar, el uso del artículo masculino "el" y el sustantivo "enfermero" (en su forma masculina) asume que la persona que ejerce la enfermería y es destacada en su campo es un hombre.

**Texto Reescrito (Neutro):**
"La persona que ocupa la nueva jefatura de ingeniería d

### Escenario 2: Control de Salida (Técnica de Guardrails / Role-Playing)
Contexto Crítico: Un reclutador necesita que el LLM genere el borrador de la descripción de un puesto de trabajo, pero la política legal prohíbe el uso de términos absolutos o discriminatorios.

Requisitos del Prompt Maestro:

1. Definición de Persona/Rol: Asigna al modelo el rol de un "Abogado Senior de Recursos Humanos" que redacta la descripción.

2. Regla de Guardrail (Control de Salida): Incluye una instrucción estricta de NO USAR ninguna de las siguientes frases prohibidas: [Cualquier término de edad: "joven", "senior" o "recién graduado"; Términos absolutos: "debes", "siempre", "garantizado"].

3. Input de la Tarea: Pídele que redacte una sección sobre los requisitos para un puesto de "Analista de Datos".

4. Output Deseado (Controlado): Un texto de la descripción del puesto que demuestre que el LLM ha acatado la regla de Guardrail.

In [5]:
# Prompt Maestro para Escenario 2: Control de Salida con Guardrails

prompt_escenario_2 = """
ROL: Eres un Abogado Senior de Recursos Humanos con 15 años de experiencia en cumplimiento legal y políticas de contratación inclusiva. Tu especialidad es redactar descripciones de puestos que cumplan estrictamente con las regulaciones laborales y antidiscriminatorias.

TAREA: Redacta la sección de "Requisitos del Puesto" para una posición de Analista de Datos.

GUARDRAILS ESTRICTOS (PROHIBICIONES ABSOLUTAS):
NO PUEDES USAR BAJO NINGUNA CIRCUNSTANCIA los siguientes términos o conceptos:

1. Términos relacionados con edad:
   - "joven", "senior", "recién graduado", "juvenil", "maduro", "experimentado en años"
   
2. Términos absolutos o coercitivos:
   - "debes", "siempre", "nunca", "garantizado", "obligatorio", "imperativo", "imprescindible"
   
3. Términos que impliquen exclusividad o discriminación:
   - Cualquier referencia a edad, género, estado civil, o características físicas

DIRECTRICES POSITIVAS:
- Usa lenguaje inclusivo y neutro
- Enfócate en competencias, habilidades y capacidades
- Usa términos como "preferible", "se valora", "buscamos", "ideal"
- Sé específico sobre las habilidades técnicas necesarias
- Mantén un tono profesional y acogedor

FORMATO REQUERIDO:
Redacta 5-7 requisitos en formato de lista que describan las cualificaciones para el puesto de Analista de Datos.

COMIENZA LA REDACCIÓN:
"""

# Llamar a la API de Gemini con el prompt de Guardrails
response_2 = model.generate_content(prompt_escenario_2)

print("="*80)
print("ESCENARIO 2: CONTROL DE SALIDA CON GUARDRAILS")
print("="*80)
print(f"\nDescripción del Puesto: Analista de Datos")
print(f"\nROL ASIGNADO: Abogado Senior de Recursos Humanos")
print("-"*80)
print(f"\nRespuesta de Gemini Pro (con Guardrails aplicados):\n")
print(response_2.text)
print("\n" + "="*80)

# Verificación de Guardrails
print("\nVERIFICACIÓN DE CUMPLIMIENTO DE GUARDRAILS:")
print("-"*80)

respuesta_texto = response_2.text.lower()

# Lista de términos prohibidos
terminos_prohibidos = [
    "joven", "senior", "recién graduado", "juvenil", "maduro",
    "debes", "siempre", "nunca", "garantizado", "obligatorio", 
    "imperativo", "imprescindible"
]

violaciones = []
for termino in terminos_prohibidos:
    if termino in respuesta_texto:
        violaciones.append(termino)

if violaciones:
    print(f"⚠️ ALERTA: Se encontraron {len(violaciones)} término(s) prohibido(s): {', '.join(violaciones)}")
else:
    print("✅ CUMPLIMIENTO VERIFICADO: No se encontraron términos prohibidos en la respuesta.")
    print("✅ Gemini Pro ha respetado todos los guardrails establecidos.")

print("="*80)

ESCENARIO 2: CONTROL DE SALIDA CON GUARDRAILS

Descripción del Puesto: Analista de Datos

ROL ASIGNADO: Abogado Senior de Recursos Humanos
--------------------------------------------------------------------------------

Respuesta de Gemini Pro (con Guardrails aplicados):

Aquí tienes la sección de "Requisitos del Puesto" para la posición de Analista de Datos, redactada con estricto apego a las directrices legales y de inclusión:

---

**Requisitos del Puesto:**

*   Habilidad probada para diseñar y ejecutar consultas SQL complejas para la extracción, transformación y manipulación de datos.
*   Conocimiento práctico de lenguajes de programación como Python o R, aplicados al análisis estadístico, la limpieza de datos y la automatización de tareas.
*   Aptitud para desarrollar visualizaciones de datos informativas e informes dinámicos utilizando herramientas de Business Intelligence (por ejemplo, Tableau, Power BI, Qlik Sense).
*   Capacidad demostrada para analizar conjuntos de datos co